# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [88]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf
from pyspark.sql.types import IntegerType
from pyspark.sql import Window
from pyspark.sql.functions import desc
from pyspark.sql.functions import sum as Fsum

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import datetime
import re
%matplotlib inline
import matplotlib.pyplot as plt

In [53]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [54]:
sparkify_data = 'mini_sparkify_event_data.json'
df = spark.read.json(sparkify_data)
df.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [55]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [56]:
df.count()

286500

In [57]:
df.describe("artist").show()

+-------+------------------+
|summary|            artist|
+-------+------------------+
|  count|            228108|
|   mean| 551.0852017937219|
| stddev|1217.7693079161374|
|    min|               !!!|
|    max| ÃÂlafur Arnalds|
+-------+------------------+



In [58]:
df.describe("userId").show()

+-------+-----------------+
|summary|           userId|
+-------+-----------------+
|  count|           286500|
|   mean|59682.02278593872|
| stddev|109091.9499991047|
|    min|                 |
|    max|               99|
+-------+-----------------+



In [59]:
df.describe("sessionId").show()

+-------+-----------------+
|summary|        sessionId|
+-------+-----------------+
|  count|           286500|
|   mean|1041.526554973822|
| stddev|726.7762634630741|
|    min|                1|
|    max|             2474|
+-------+-----------------+



In [60]:
df = df.dropna(how="any", subset=["userId","sessionId"])

In [61]:
df.count()

286500

In [62]:
df.select("userId").dropDuplicates().sort("userId").show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



In [63]:
df.filter(df["userId"] == "").count()

8346

In [64]:
# all pages
df.select("page").dropDuplicates().collect()

[Row(page='Cancel'),
 Row(page='Submit Downgrade'),
 Row(page='Thumbs Down'),
 Row(page='Home'),
 Row(page='Downgrade'),
 Row(page='Roll Advert'),
 Row(page='Logout'),
 Row(page='Save Settings'),
 Row(page='Cancellation Confirmation'),
 Row(page='About'),
 Row(page='Submit Registration'),
 Row(page='Settings'),
 Row(page='Login'),
 Row(page='Register'),
 Row(page='Add to Playlist'),
 Row(page='Add Friend'),
 Row(page='NextSong'),
 Row(page='Thumbs Up'),
 Row(page='Help'),
 Row(page='Upgrade'),
 Row(page='Error'),
 Row(page='Submit Upgrade')]

In [65]:
## blank userId visited pages
df.filter(df.userId == "").select("page").dropDuplicates().collect()

[Row(page='Home'),
 Row(page='About'),
 Row(page='Submit Registration'),
 Row(page='Login'),
 Row(page='Register'),
 Row(page='Help'),
 Row(page='Error')]

Blank userId are guest who just look around the Sparkiy, so we drop them from the dataset

In [66]:
df = df.filter(df.userId != "")
df.count()

278154

In [67]:
df.select("sessionId").dropDuplicates().sort("sessionId").show()

+---------+
|sessionId|
+---------+
|        1|
|        2|
|        3|
|        4|
|        5|
|        6|
|        7|
|        8|
|        9|
|       10|
|       11|
|       12|
|       13|
|       15|
|       16|
|       17|
|       18|
|       19|
|       20|
|       21|
+---------+
only showing top 20 rows



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [68]:
df.filter("page = 'Cancellation Confirmation'").head()

Row(artist=None, auth='Cancelled', firstName='Adriel', gender='M', itemInSession=104, lastName='Mendoza', length=None, level='paid', location='Kansas City, MO-KS', method='GET', page='Cancellation Confirmation', registration=1535623466000, sessionId=514, song=None, status=200, ts=1538943990000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"', userId='18')

In [69]:
## we flag the cancellation event 1
cancel_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
df = df.withColumn("cancel", cancel_event("page"))
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', cancel=0)

Since Cancellation can happen to paid and free users, we flag these users 1 on all event records before their cancellation and 0 otherwise by creating Churn column.

In [70]:
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
df = df.withColumn("Churn", Fsum("cancel").over(windowval))

In [71]:
df.select(["userId", "firstname", "ts", "page","method","sessionId","song", "level", "Churn"]).where(df.userId == "100014").sort("ts").collect()

[Row(userId='100014', firstname='Rodrigo', ts=1539177136000, page='NextSong', method='PUT', sessionId=37, song='Welcome To The World of The Plastic Beach (Feat. Snoop Dogg and Hypnotic Brass Ensemble)', level='paid', Churn=1),
 Row(userId='100014', firstname='Rodrigo', ts=1539177351000, page='NextSong', method='PUT', sessionId=37, song='Odessa', level='paid', Churn=1),
 Row(userId='100014', firstname='Rodrigo', ts=1539177476000, page='NextSong', method='PUT', sessionId=37, song='Amapola', level='paid', Churn=1),
 Row(userId='100014', firstname='Rodrigo', ts=1539177698000, page='NextSong', method='PUT', sessionId=37, song='Exogenesis: Symphony Part 1 [Overture]', level='paid', Churn=1),
 Row(userId='100014', firstname='Rodrigo', ts=1539177705000, page='Help', method='GET', sessionId=37, song=None, level='paid', Churn=1),
 Row(userId='100014', firstname='Rodrigo', ts=1539177956000, page='NextSong', method='PUT', sessionId=37, song='Sehr kosmisch', level='paid', Churn=1),
 Row(userId='100

Exploratory Data Analysis on groups who stayed and churned

In [72]:
# About 16% churn rate
df.groupby("Churn").count().show()

+-----+------+
|Churn| count|
+-----+------+
|    0|233290|
|    1| 44864|
+-----+------+



In [73]:
song_event = udf(lambda x: 1 if x == "NextSong" else 0, IntegerType())
df = df.withColumn("listen", song_event("page"))

On average, users who stayed listened to 676 songs and users who churned listened to 431 songs

In [74]:
df.filter("Churn == 0 and listen == 1").groupby("userId").count().agg({"count":"avg"}).show()

+------------------+
|        avg(count)|
+------------------+
|1108.1734104046243|
+------------------+



In [75]:
df.filter("Churn == 1 and listen == 1").groupby("userId").count().agg({"count":"avg"}).show()

+-----------------+
|       avg(count)|
+-----------------+
|699.8846153846154|
+-----------------+



In [76]:
thumbsdown_event = udf(lambda x: 1 if x == "Thumbs Down" else 0, IntegerType())
df = df.withColumn("ThumbsDown", thumbsdown_event("page"))
help_event = udf(lambda x: 1 if x == "Help" else 0, IntegerType())
df = df.withColumn("Help", help_event("page"))
error_event = udf(lambda x: 1 if x == "Error" else 0, IntegerType())
df = df.withColumn("Error", error_event("page"))
ad_event = udf(lambda x: 1 if x == "Roll Advert" else 0, IntegerType())
df = df.withColumn("Ad", ad_event("page"))


In [77]:
df.filter("Churn == 0 and ThumbsDown == 1").groupby("userId").count().agg({"count":"avg"}).show()

+------------------+
|        avg(count)|
+------------------+
|13.225806451612904|
+------------------+



In [78]:
df.filter("Churn == 1 and ThumbsDown == 1").groupby("userId").count().agg({"count":"avg"}).show()

+------------------+
|        avg(count)|
+------------------+
|10.333333333333334|
+------------------+



In [79]:
df.filter("Churn == 0 and Help == 1").groupby("userId").count().agg({"count":"avg"}).show()

+-----------------+
|       avg(count)|
+-----------------+
|8.154362416107382|
+-----------------+



In [80]:
df.filter("Churn == 1 and Help == 1").groupby("userId").count().agg({"count":"avg"}).show()

+-----------------+
|       avg(count)|
+-----------------+
|5.558139534883721|
+-----------------+



In [81]:
df.filter("Churn == 0 and Error == 1").groupby("userId").count().agg({"count":"avg"}).show()

+------------------+
|        avg(count)|
+------------------+
|2.3157894736842106|
+------------------+



In [82]:
df.filter("Churn == 1 and Error == 1").groupby("userId").count().agg({"count":"avg"}).show()

+------------------+
|        avg(count)|
+------------------+
|1.4545454545454546|
+------------------+



In [83]:
df.filter("Churn == 0 and Ad == 1").groupby("userId").count().agg({"count":"avg"}).show()

+-----------------+
|       avg(count)|
+-----------------+
|18.89171974522293|
+-----------------+



In [84]:
df.filter("Churn == 1 and Ad == 1").groupby("userId").count().agg({"count":"avg"}).show()

+----------+
|avg(count)|
+----------+
|     19.34|
+----------+



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [85]:
windowcum = Window.partitionBy("userId").orderBy("ts").rangeBetween(Window.unboundedPreceding, 0)
df = df.withColumn("numSong", Fsum("listen").over(windowcum))
df = df.withColumn("numThumbsDown", Fsum("ThumbsDown").over(windowcum))
df = df.withColumn("numHelp", Fsum("Help").over(windowcum))
df = df.withColumn("numError", Fsum("Error").over(windowcum))
df = df.withColumn("numAd", Fsum("Ad").over(windowcum))

In [86]:
df.select(["ts", "page","song", "numSong","numThumbsDown","numHelp","numAd", "Churn"]).where(df.userId == "100014").sort("ts").collect()

[Row(ts=1539177136000, page='NextSong', song='Welcome To The World of The Plastic Beach (Feat. Snoop Dogg and Hypnotic Brass Ensemble)', numSong=1, numThumbsDown=0, numHelp=0, numAd=0, Churn=1),
 Row(ts=1539177351000, page='NextSong', song='Odessa', numSong=2, numThumbsDown=0, numHelp=0, numAd=0, Churn=1),
 Row(ts=1539177476000, page='NextSong', song='Amapola', numSong=3, numThumbsDown=0, numHelp=0, numAd=0, Churn=1),
 Row(ts=1539177698000, page='NextSong', song='Exogenesis: Symphony Part 1 [Overture]', numSong=4, numThumbsDown=0, numHelp=0, numAd=0, Churn=1),
 Row(ts=1539177705000, page='Help', song=None, numSong=4, numThumbsDown=0, numHelp=1, numAd=0, Churn=1),
 Row(ts=1539177956000, page='NextSong', song='Sehr kosmisch', numSong=5, numThumbsDown=0, numHelp=1, numAd=0, Churn=1),
 Row(ts=1539178611000, page='NextSong', song='Moghram Ya Lail', numSong=6, numThumbsDown=0, numHelp=1, numAd=0, Churn=1),
 Row(ts=1539178914000, page='NextSong', song='AtrÃ\x83Â©vete te te', numSong=7, numThu

In [89]:
indexer = StringIndexer(inputCol="page", outputCol="pageLabel")
df = indexer.fit(df).transform(df)
df.head()

Row(artist='Sleeping With Sirens', auth='Logged In', firstName='Darianna', gender='F', itemInSession=0, lastName='Carpenter', length=202.97098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Captain Tyin Knots VS Mr Walkway (No Way)', status=200, ts=1539003534000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', cancel=0, Churn=0, listen=1, ThumbsDown=0, Help=0, Error=0, Ad=0, numSong=1, numThumbsDown=0, numHelp=0, numError=0, numAd=0, pageLabel=0.0)

In [97]:
get_date = udf(lambda x: str(datetime.datetime.fromtimestamp(x / 1000.0).date()))
df = df.withColumn("date",get_date(df.ts))

[Row(ts=1539177136000, page='NextSong', song='Welcome To The World of The Plastic Beach (Feat. Snoop Dogg and Hypnotic Brass Ensemble)', numSong=1, numThumbsDown=0, numHelp=0, numAd=0, date='2018-10-10'),
 Row(ts=1539177351000, page='NextSong', song='Odessa', numSong=2, numThumbsDown=0, numHelp=0, numAd=0, date='2018-10-10'),
 Row(ts=1539177476000, page='NextSong', song='Amapola', numSong=3, numThumbsDown=0, numHelp=0, numAd=0, date='2018-10-10'),
 Row(ts=1539177698000, page='NextSong', song='Exogenesis: Symphony Part 1 [Overture]', numSong=4, numThumbsDown=0, numHelp=0, numAd=0, date='2018-10-10'),
 Row(ts=1539177705000, page='Help', song=None, numSong=4, numThumbsDown=0, numHelp=1, numAd=0, date='2018-10-10'),
 Row(ts=1539177956000, page='NextSong', song='Sehr kosmisch', numSong=5, numThumbsDown=0, numHelp=1, numAd=0, date='2018-10-10'),
 Row(ts=1539178611000, page='NextSong', song='Moghram Ya Lail', numSong=6, numThumbsDown=0, numHelp=1, numAd=0, date='2018-10-10'),
 Row(ts=15391789

In [101]:
windowall = Window.partitionBy("userId","date").orderBy("ts").rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df = df.withColumn("numSongPerDay", Fsum("listen").over(windowall))

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.